In [385]:
from scipy import *
from scipy.misc import derivative
from scipy.special import *
from tokenize import TokenError
import bqplot.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, Latex, HTML, Math
import sympy as sym
from sympy.parsing.sympy_parser import parse_expr

In [386]:
x = sym.Symbol('x')

## Derivative Explorer
Use the entry widgets below the plots to examine the connection between the slope of a given function and the value of its derivative.

The function can be changed by typing a new function into the 'Enter function' box hitting the Return/Enter key.  The function must be described in terms of the variable 'x'. You must use an '\*' to indicate multiplication.  Powers use '\*\*'.

The domain can be changed similarly.

The position at which the derivative is calculated can be moved back and forth using the labelled slider widget.

### Try:
(1) Enter the function 'sin(3\*x) \* exp(-x/2)'

(2) Try tanh(x)

#### Bonus (that won't be shared with students)
This application knows all of the special functions from scipy.special:

Try:  'eval_hermite(3,x)' to get the third Hermite polynomial.  The title will look funky, but the plots are correct.

Try:  Set the domain to 0.1 to 3.0, use function 'y0(x)' to get Bessel function of the second kind of order 0.

So see other functions, look at [Scipy Special Functions](https://docs.scipy.org/doc/scipy/reference/special.html).  To work, the function can only be a function of one real variable (you can pass integer values, as in the Hermite polynomial example), and must return one real value (not a tuple of values, as for example, with the Airy function).

#### Future:
I'm hoping to build a similar project to demonstrate numerical integration for students, showing explicitly the little shapes used (*e.g.* rectangles, trapezoids, *etc.*) and comparing estimates with accurate values for arbitrary functions.

I've thought about adding a third plot pane to this page showing the second derivative, perhaps superimposing an oscullating parabola to show the curvature on the main plot....  I still might...  maybe make it optional with a check box...

In [390]:
def set_function(strFunc):
    try:
        local_func = lambda x: eval(strFunc)
        def local_deriv(x):
            return around(derivative(local_func, x, dx=1e-6), 4)
        f = parse_expr(strFunc)
    except TokenError:
        print("malformed function")
        return None
    return local_func, local_deriv, f

strFunc = "1+x**2"
f, df, fstr = set_function(strFunc)
sym_func = sym.latex(fstr)


label_layout = widgets.Layout(display='flex', justify_content='center', width = "70%", 
                              align_items = 'center', border = 'none')
fig_layout = widgets.Layout(height = "350px", width = "350px")
fig_margin_dict = dict(top=30, bottom = 30, left = 20, right = 10)

label = widgets.Label(r"$f(x) =  {}$".format(sym.latex(1+x**2)), layout = label_layout)
title_format = "<h2><font color='blue'>$f(x) = {}$</font></h2>"
title_label = widgets.HTMLMath(title_format.format(sym_func),
                              layout = label_layout)

# create initial plots
low = -3
high = 3
xpoint = 0
nPts = 200  # number of points in domain
xx = linspace(low, high, nPts)
yy = f(xx)

# Main figure
main_fig = plt.figure(title="function", layout = fig_layout, fig_margin = fig_margin_dict)
function_curve = plt.plot(xx, yy)
ylimits = plt.ylim(min(yy), max(yy))
ypoint = f(xpoint)
point = plt.plot([xpoint], [ypoint], marker='circle', colors = ['red'])

# add derivative line to main figure:
m = df(xpoint)
b = ypoint - xpoint * m
deriv_line = lambda x: m*x + b
df_line = plt.plot(xx, deriv_line(xx), colors=['green'])

# create point slider
point_slider = widgets.FloatSlider(value = 0.5, min = low, max = high, step = 0.01, 
                                  readout = True, readout_format = '.2f', continuous_update = True)
point_label = widgets.Label("Position of red dot: ")
point_box = widgets.HBox([point_label, point_slider])

# Create derivative plot
deriv_fig = plt.figure(title="derivative", layout = fig_layout, fig_margin=fig_margin_dict)
deriv_curve = plt.plot(xx, df(xx), fig = deriv_fig)
dy_point = df(xpoint)
deriv_point = plt.plot([xpoint], [dy_point], colors = ['red'], marker = 'circle')
deriv_value = plt.plot(xx, [dy_point]*nPts, colors = ['green'])

# plot box
plot_box = widgets.HBox([main_fig, deriv_fig])

# function entry
func_label = widgets.Label("Enter function: ")
func_entry = widgets.Text(strFunc, continuous_update = False)
func_box = widgets.HBox([func_label, func_entry])

# domain entry
dom_layout = widgets.Layout(width = '75px')
dom_label = widgets.Label("Enter domain: ")
dom_low = widgets.FloatText(value = -3.0, layout = dom_layout)
dom_high = widgets.FloatText(value = 3.0, layout = dom_layout)
out_layout = widgets.Layout(border="blue solid 2px")
out_str = "The derivative at {} is {}"
output = widgets.Text(value = out_str.format(xpoint, dy_point), layout = out_layout)
dom_box = widgets.HBox([dom_label, dom_low, dom_high, output])

# callback functions
def update_plot(change):
    #if change['owner'] == func_entry:
    strFunc = func_entry.value
    f, df, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    low = dom_low.value
    high = dom_high.value
    point_slider.min = low
    point_slider.max = high
    x = linspace(dom_low.value, dom_high.value, nPts)
    y = f(x)
    # fix y-axis limits:
    ylimits.min = min(y)
    ylimits.max = max(y)
    
    # update main plot
    function_curve.x = x
    function_curve.y = y
    df_line.x = x
    
    xpoint = point_slider.value
    ypoint = f(xpoint)
    m = df(xpoint)
    b = ypoint - xpoint * m
    df_line.y = m*x + b
    point.x = [xpoint]
    point.y = [f(xpoint)]
    
    # update derivative plot
    deriv_curve.x = x
    deriv_curve.y = df(x)
    deriv_point.x = [xpoint]
    deriv_point.y = [df(xpoint)]
    deriv_value.x = x
    deriv_value.y = [df(xpoint)]*nPts
    
    # update output
    output.value = out_str.format(xpoint, df(xpoint))
    
def update_function(change):
    global f, df
    strFunc = func_entry.value
    f, df, fstr = set_function(strFunc)
    title_label.value = title_format.format(sym.latex(fstr))
    update_plot("called from update_function")
    
point_slider.observe(update_plot, names = 'value')
func_entry.observe(update_plot, names = 'value')
dom_low.observe(update_plot, names = 'value')
dom_high.observe(update_plot, names = 'value')

widgets.VBox([title_label, plot_box, point_box, func_box, dom_box])

TypeError: bernoulli() takes 1 positional argument but 2 were given

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

C:\Anaconda3\lib\site-packages\scipy\misc\common.py:119: RuntimeWarning: invalid value encountered in double_scalars
  val += weights[k]*func(x0+(k-ho)*dx,*args)
C:\Anaconda3\lib\site-packages\scipy\misc\common.py:119: RuntimeWarning: invalid value encountered in multiply
  val += weights[k]*func(x0+(k-ho)*dx,*args)


In [370]:
df

<function __main__.set_function.<locals>.local_deriv(x)>

In [338]:
df(0)

TypeError: 'Add' object is not callable

In [339]:
f

<function __main__.alt_set_function.<locals>.<lambda>(x)>

In [402]:
y0(0)

-inf

In [379]:
import scipy.special
dir(scipy.special)

['SpecialFunctionError',
 'SpecialFunctionWarning',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_comb',
 '_ellip_harm',
 '_ellip_harm_2',
 '_logsumexp',
 '_spherical_bessel',
 '_ufuncs',
 '_ufuncs_cxx',
 'absolute_import',
 'agm',
 'ai_zeros',
 'airy',
 'airye',
 'assoc_laguerre',
 'basic',
 'bdtr',
 'bdtrc',
 'bdtri',
 'bdtrik',
 'bdtrin',
 'bei',
 'bei_zeros',
 'beip',
 'beip_zeros',
 'ber',
 'ber_zeros',
 'bernoulli',
 'berp',
 'berp_zeros',
 'bessel_diff_formula',
 'besselpoly',
 'beta',
 'betainc',
 'betaincinv',
 'betaln',
 'bi_zeros',
 'binom',
 'boxcox',
 'boxcox1p',
 'btdtr',
 'btdtri',
 'btdtria',
 'btdtrib',
 'c_roots',
 'cbrt',
 'cg_roots',
 'chdtr',
 'chdtrc',
 'chdtri',
 'chdtriv',
 'chebyc',
 'chebys',
 'chebyt',
 'chebyu',
 'chndtr',
 'chndtridf',
 'chndtrinc',
 'chndtrix',
 'clpmn',
 'comb',
 'cosdg',
 'cosm1',
 'cotdg',
 'dawsn',
 'digamma',
 'diric',
 'division',
 'ellip_

In [380]:
hermite(xpoint)

poly1d([1.])

In [400]:
info(jv)

jv(x1, x2, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])

jv(v, z)

Bessel function of the first kind of real order and complex argument.

Parameters
----------
v : array_like
    Order (float).
z : array_like
    Argument (float or complex).

Returns
-------
J : ndarray
    Value of the Bessel function, :math:`J_v(z)`.

Notes
-----
For positive `v` values, the computation is carried out using the AMOS
[1]_ `zbesj` routine, which exploits the connection to the modified
Bessel function :math:`I_v`,

.. math::
    J_v(z) = \exp(v\pi\imath/2) I_v(-\imath z)\qquad (\Im z > 0)

    J_v(z) = \exp(-v\pi\imath/2) I_v(\imath z)\qquad (\Im z < 0)

For negative `v` values the formula,

.. math:: J_{-v}(z) = J_v(z) \cos(\pi v) - Y_v(z) \sin(\pi v)

is used, where :math:`Y_v(z)` is the Bessel function of the second
kind, computed using the AMOS routine `zbesy`.  Note that the second
term is exactly zero for integer `v`; to improve accuracy 

In [391]:
info(bernoulli)

 bernoulli(n)

Bernoulli numbers B0..Bn (inclusive).

References
----------
.. [1] Zhang, Shanjie and Jin, Jianming. "Computation of Special
       Functions", John Wiley and Sons, 1996.
       https://people.sc.fsu.edu/~jburkardt/f_src/special_functions/special_functions.html
